In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

TODO: Go through this in more detail to see how the two implementations vary, and why they give different results.

Note: Figured out where the input is stored by default...  Note the directory structure.

Since I used a dataset in Kaggle with the same info but slightly different format I had to edit some details to get this all to work.

In [10]:
!ls /kaggle/input/iris-dataset

iris.csv


In [11]:
!head /kaggle/input/iris-dataset/iris.csv

sepal_length,sepal_width,petal_length,petal_width,species
5.1,3.5,1.4,0.2,setosa
4.9,3.0,1.4,0.2,setosa
4.7,3.2,1.3,0.2,setosa
4.6,3.1,1.5,0.2,setosa
5.0,3.6,1.4,0.2,setosa
5.4,3.9,1.7,0.4,setosa
4.6,3.4,1.4,0.3,setosa
5.0,3.4,1.5,0.2,setosa
4.4,2.9,1.4,0.2,setosa


In [5]:
import numpy as np
import random


# Perceptron with numpy + pandas
# The guy is great, will definetly give a shoutout
class Perceptron():
    '''
        Perceptron Learning Algorithm that can be train using a 
        fit and predict methodology with numpy
    '''
    
    def __init__(self):
        self.weights = []
        
    def fit(self, X, y, learning_rate = 0.01, num_iteration = 100):
        
        (num_row, num_feature) = X.shape
        
        # Randomly initalize the weights
        self.weights = np.random.rand(num_feature+1) 

        # Launch the training algorithm
        for i in range(num_iteration):
            
            # Stochastic Gradient Descent
            r_i = random.randint(0,num_row-1)
            row = X[r_i,:] # take the random sample from the dataset
            yhat = self.predict(row)
            error = (y[r_i] - yhat) # estimate of the gradient
            self.weights[0] = self.weights[0] + learning_rate*error*1 # first weight one is the bias

            # Update all parameters after bias
            for f_i in range(num_feature):
                self.weights[f_i] = self.weights[f_i] + learning_rate*error*row[f_i]
                
            if i % 100 == 0:
                total_error = 0
                for r_i in range(num_row):
                    row = X[r_i,:]
                    yhat = self.predict(row)
                    error = (y[r_i] - yhat)
                    total_error = total_error + error**2
                mean_error = total_error/num_row
                print(f"Iteration {i} with error = {mean_error}")
        
    def predict(self, row):
            
        # The activation start with the bias at weights == 0
        activation = self.weights[0]
        
        # We iterate over the weights and the features in the given row
        for weight, feature in zip(self.weights[1:], row):
            activation = activation + weight*feature
            
        # Heaviside Step Function Activation
        if activation >= 0.0:
            return 1.0
        return 0.0

In [18]:
import pandas as pd

# Data sets
df = pd.read_csv('/kaggle/input/iris-dataset/iris.csv')

df.describe(include='all')

,sepal_length,sepal_width,petal_length,petal_width,species
count,150.000000,150.000000,150.000000,150.000000,150
unique,NaN,NaN,NaN,NaN,3
top,NaN,NaN,NaN,NaN,setosa
freq,NaN,NaN,NaN,NaN,50
mean,5.843333,3.054000,3.758667,1.198667,NaN
std,0.828066,0.433594,1.764420,0.763161,NaN
min,4.300000,2.000000,1.000000,0.100000,NaN
25%,5.100000,2.800000,1.600000,0.300000,NaN
50%,5.800000,3.000000,4.350000,1.300000,NaN
75%,6.400000,3.300000,5.100000,1.800000,NaN


In [24]:
# Do a one hot encoding
df = pd.get_dummies(df,prefix='species')
X = df[['sepal_length','sepal_width','petal_length','petal_width']]
X = X.to_numpy()

df.describe(include='all')

,sepal_length,sepal_width,petal_length,petal_width,variety_setosa,variety_versicolor,variety_virginica
count,150.000000,150.000000,150.000000,150.000000,150,150,150
unique,NaN,NaN,NaN,NaN,2,2,2
top,NaN,NaN,NaN,NaN,False,False,False
freq,NaN,NaN,NaN,NaN,100,100,100
mean,5.843333,3.054000,3.758667,1.198667,NaN,NaN,NaN
std,0.828066,0.433594,1.764420,0.763161,NaN,NaN,NaN
min,4.300000,2.000000,1.000000,0.100000,NaN,NaN,NaN
25%,5.100000,2.800000,1.600000,0.300000,NaN,NaN,NaN
50%,5.800000,3.000000,4.350000,1.300000,NaN,NaN,NaN
75%,6.400000,3.300000,5.100000,1.800000,NaN,NaN,NaN


In [26]:

y = df.loc[:, 'variety_versicolor']
y = y.to_numpy()

# Shuffle the two dataset in unison
perm = np.random.permutation(len(X))
X = X[perm]
y = y[perm]

clf = Perceptron()
clf.fit(X,y, num_iteration = 1000)

Iteration 0 with error = 0.6666666666666666
Iteration 100 with error = 0.58
Iteration 200 with error = 0.5
Iteration 300 with error = 0.38666666666666666
Iteration 400 with error = 0.3466666666666667
Iteration 500 with error = 0.3933333333333333
Iteration 600 with error = 0.35333333333333333
Iteration 700 with error = 0.5666666666666667
Iteration 800 with error = 0.6333333333333333
Iteration 900 with error = 0.37333333333333335


In [27]:
import random

class Perceptron():
    '''
        Perceptron Learning Algorithm that can be train using a 
        fit and predict methodology, without any library
    '''
    
    def __init__(self):
        self.weights = []
        
    def fit(self, X, y, learning_rate = 0.01, num_iteration = 100):
        
        num_row = len(X)
        num_feature = len(X[0]) # Here we assume that we have a rectangular matrix
        
        # Randomly initalize the weights
        for i in range(num_feature+1):
            self.weights.append(random.uniform(0,1))
        
        # Launch the training algorithm
        
        for i in range(num_iteration):
            
            # Stochastic Gradient Descent
            r_i = random.randint(0,num_row-1)
            row = X[r_i]
            yhat = self.predict(row)
            error = (y[r_i] - yhat)
            self.weights[0] = self.weights[0] + learning_rate*error

            for f_i in range(num_feature):
                self.weights[f_i] = self.weights[f_i] + learning_rate*error*row[f_i]
                
            if i % 100 == 0:
                total_error = 0
                for r_i in range(num_row):
                    row = X[r_i]
                    yhat = self.predict(row)
                    error = (y[r_i] - yhat)
                    total_error = total_error + error**2
                mean_error = total_error/num_row
                print(f"Iteration {i} with error = {mean_error}")
        
    def predict(self, row):
            
        # The activation start with the bias at weights == 0
        activation = self.weights[0]
        
        # We iterate over the weights and the features in the given row
        for weight, feature in zip(self.weights[1:], row):
            activation = activation + weight*feature
            
        if activation >= 0.0:
            return 1.0
        return 0.0

In [29]:
import csv

def permute_together(X,y):
    '''
        Helper function to permute (shuffle) a matrix and a vector together
    '''
    
    perm_X = []
    perm_y = []
    while len(X) != 0 and len(y) != 0:
        
        perm_id = random.randint(0,len(X)-1)
        perm_X.append(X.pop(perm_id))
        perm_y.append(y.pop(perm_id))
        
    return (perm_X, perm_y)

class DataFrame():
    '''
        Simple dataframe to mimick the pandas library
    '''
    def __init__(self):
        self.header = [];
        self.X = []
        self.y = []
        
    def clean_string(self,string):
        '''
            Dummy function to clean up the iris dataset from (")
        '''
        return string.replace('"', '')
    

    def get_encoded_labels(self, target):
        '''
            Encode with 1 or 0 the y vector if it match our target variable
        '''
        labels = []
        for label in self.y:
            if label == target:
                labels.append(1)
            else:
                labels.append(0)
        return labels
    
    def read_csv(self, filename):
        '''
            Read the iris dataset CSV file and populate the header, the X and the y variables
            needed for the perceptron
        '''
        with open(filename, newline='', encoding="utf-8-sig") as csvfile:
            csvreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
            
            index = -1
            for row in csvreader:
                
                # We have the header
                if index == -1:
                    self.header = [self.clean_string(s) for s in row[0].split(',')]
                    index += 1
                    continue
                
                # Here is the data
                x = []
                target = None
                data = row[0].split(',')
                for i in range(len(data)-1):
                    x.append(float(data[i]))
                
                # Last item in the csv will be the target
                self.y.append(self.clean_string(data[len(data)-1]))
                self.X.append(x)
                
                index += 1
        
                

# Data sets
df = DataFrame()
df.read_csv('/kaggle/input/iris-dataset/iris.csv')

X = df.X
y = df.get_encoded_labels('Versicolor') # encoding for 0 and 1

# Shuffle the two dataset in unison
X,y = permute_together(X,y)

clf = Perceptron()
clf.fit(X,y, num_iteration = 1000)

Iteration 0 with error = 1.0
Iteration 100 with error = 0.0
Iteration 200 with error = 0.0
Iteration 300 with error = 0.0
Iteration 400 with error = 0.0
Iteration 500 with error = 0.0
Iteration 600 with error = 0.0
Iteration 700 with error = 0.0
Iteration 800 with error = 0.0
Iteration 900 with error = 0.0
